In [1]:
from langchain.schema import Document
import json


# 파일 로드 및 doc 화
file_paths = {
    "term": "./metadata/term.json",
    "load_traffic_law": "./metadata/load_traffic_law.json",
    "modifier": "./metadata/modifier.json",
    "car_case": "./metadata/car_to_car.json",
    "precedent": "./metadata/precedent.json"
}

# 교통사고 케이스용 필드 상수
CASE_ID = "사건 ID"
CASE_TITLE = "사건 제목"
CASE_SITUATION = "사고상황"
BASE_RATIO = "기본 과실비율"
MODIFIERS = "케이스별 과실비율 조정예시"
LAW_REFERENCES = "관련 법규"
PRECEDENT = "참고 판례"
REASON = "기본 과실비율 해설"

# JSON 로드 함수
def load_json(path):
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)

# 리스트형 JSON 변환 (term, modifier, law_meta)
def convert_list_to_documents(data_list, doc_type):
    return [
        Document(page_content=json.dumps(item, ensure_ascii=False), metadata={"type": doc_type})
        for item in data_list
    ]

def convert_precedent_documents(data_list):
    return [
        Document(
            page_content=f"{item['court']} {item['case_id']} : {item['content']}",
            metadata={
                "court": item["court"],
                "case_id": item["case_id"],
            }
        ) for item in data_list
    ]

def convert_term_documents(data_list):
    return [
        Document(
            page_content=f"{item['term']} : {item['desc']}",
            metadata={
                "term": item["term"]
            }
        ) for item in data_list
    ]

def convert_car_case_documents(data_list):
    documents = []

    def safe_value(value):
        if isinstance(value, list):
            return ", ".join(map(str, value))
        elif isinstance(value, dict):
            return json.dumps(value, ensure_ascii=False)
        elif value is None:
            return ""  # null도 허용 안 되므로 빈 문자열로 처리
        else:
            return str(value)

    for item in data_list:
        if not isinstance(item, dict):
            continue

        # page_content는 원본 전체 JSON 문자열
        content = json.dumps(item, ensure_ascii=False)

        # 기본 과실비율 해설이 리스트일 수 있음 → 문자열로 병합
        reason = item.get(REASON)
        if isinstance(reason, list):
            reason = "\n".join(map(str, reason))

        metadata = {
            "type": "car_case",
            "id": safe_value(item.get(CASE_ID)),
            "title": safe_value(item.get(CASE_TITLE)),
            "situation": safe_value(item.get(CASE_SITUATION)),
            "base_ratio": safe_value(item.get(BASE_RATIO)),
            "modifiers": safe_value(item.get(MODIFIERS)),
            "load_traffic_law": safe_value(item.get(LAW_REFERENCES)),
            "precedent": safe_value(item.get(PRECEDENT)),
            "reason": safe_value(reason)
        }

        documents.append(Document(page_content=content, metadata=metadata))
    return documents

# 도로교통법 law JSON → 문서화
def convert_law_json_to_documents(data_dict):
    documents = []

    def normalize(item):
        return json.dumps(item, ensure_ascii=False) if isinstance(item, dict) else str(item)

    for law_name, content in data_dict.items():
        if isinstance(content, dict):
            for clause, text in content.items():
                lines = [normalize(x) for x in (text if isinstance(text, list) else [text])]
                full_text = f"{law_name} {clause}\n" + "\n".join(lines)
                documents.append(Document(page_content=full_text, metadata={"type": "load_traffic_law"}))
        else:
            lines = [normalize(x) for x in (content if isinstance(content, list) else [content])]
            full_text = f"{law_name}\n" + "\n".join(lines)
            documents.append(Document(page_content=full_text, metadata={"type": "load_traffic_law"}))
    
    return documents
    

import random
json_precedent = load_json(file_paths["precedent"])
random_precedent = random.sample(json_precedent, 10)
# for precedent in random_precedent:
#     print(precedent['court'])
#     print(precedent['case_id'])

# 문서화 실행
term_docs = convert_term_documents(load_json(file_paths["term"]))
modifier_docs = convert_list_to_documents(load_json(file_paths["modifier"]), "modifier")
precedent_docs = convert_precedent_documents(load_json(file_paths["precedent"]))
car_case_docs = convert_car_case_documents(load_json(file_paths["car_case"]))
load_traffic_law_docs = convert_law_json_to_documents(load_json(file_paths["load_traffic_law"]))


# 전체 문서 리스트
all_docs = term_docs + modifier_docs + car_case_docs + precedent_docs + load_traffic_law_docs

from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings


from langchain.vectorstores import Chroma  # persist 지원
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 1. 청크 크기 조정 (500~1000 권장)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    length_function=len,
    separators=["\n\n", "\n", "(?<=\\. )", " ", ""],
    is_separator_regex=True,
)

# 2. 문서 분할
all_splits = text_splitter.split_documents(all_docs)

embedding_model = OpenAIEmbeddings()

# 4. Chroma DB에 배치 처리로 저장
batch_size = 100  # 한 번에 처리할 청크 수
vectorstore = Chroma.from_documents(
    documents=all_splits[:batch_size],  # 첫 배치
    embedding=embedding_model,
    persist_directory="./vectordb"
)

# 남은 청크를 순차적으로 추가 
for i in range(0, len(all_splits), batch_size):
    try:
        batch = all_splits[i:i+batch_size]
        vectorstore.add_documents(batch)
        vectorstore.persist()  # 매 배치 후 즉시 저장
    except Exception as e:
        print(f"배치 {i}~{i+batch_size} 저장 실패: {e}")

vectorstore.persist()

C:\Users\Playdata\AppData\Local\Temp\ipykernel_5164\1998050127.py:171: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()  # 매 배치 후 즉시 저장


In [5]:
# 기능 분류 및 라우팅 처리 + 사고 상황 기반 과실비율 판단 포함
import re
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain.schema import Document
import json

# 함수 정의 (사고 상황 입력시)
def assess_accident_fault(user_input: str, all_docs: list) -> str:
    import json
    import numpy as np
    import re
    from sentence_transformers import SentenceTransformer
    from langchain.schema import Document
    from langchain.prompts import PromptTemplate
    from langchain.chains import LLMChain
    from langchain_openai import ChatOpenAI

    # car_case 문서 필터링 및 사고상황 추출
    case_docs = [doc for doc in all_docs if doc.metadata.get("type") == "car_case"]
    case_texts = [doc.metadata.get("situation", "") for doc in case_docs if doc.metadata.get("situation")]

    # ko-sbert 임베딩
    embed_model = SentenceTransformer("jhgan/ko-sbert-nli")
    case_embeddings = embed_model.encode(case_texts)

    # 사용자 입력
    query_embedding = embed_model.encode([user_input])[0]

    # 코사인 유사도 계산 및 Top-3 추출
    cos_similarities = np.dot(case_embeddings, query_embedding) / (
        np.linalg.norm(case_embeddings, axis=1) * np.linalg.norm(query_embedding)
    )
    top_k_idx = np.argsort(cos_similarities)[-3:][::-1]
    top_candidates = [case_docs[i] for i in top_k_idx]

    # 판례 요약 출력
    def summarize(doc, idx):
        return f"{idx+1}. 사건 ID: {doc.metadata.get('id')}\n사고상황: {doc.metadata.get('situation')}"

    case_summaries = "\n\n".join([summarize(doc, i) for i, doc in enumerate(top_candidates)])

    # GPT - 사건ID 선택(3개 중에 하나 판단)
    selection_prompt = PromptTemplate(
        input_variables=["user_input", "case_summaries"],
        template="""
    [사용자 입력 사고 상황]
    {user_input}

    [후보 판례 3건]
    {case_summaries}

    위 3건 중, 사고의 전개 구조(예: 직진 vs 좌회전, 도로 외 장소에서 진입, 교차로 내 진입 여부 등)가 사용자 상황과 가장 유사한 **사건 ID** 하나를 선택하세요.

    반드시 다음 기준을 고려하세요:
    - 차량들의 위치와 진입 경로가 유사한가?
    - 사고 발생 지점과 방향이 유사한가?
    - 각 차량의 신호·우선권 상황이 유사한가?|
    - 도로 구조(교차로, 신호 유무, 도로 외 장소 등)가 유사한가?

    출력 형식 (고정):
    - 사건 ID: 차XX-X
    - 판단 근거: (선택한 이유. 단순 유사성이 아니라, 어떤 지점이 유사했는지 명확히 설명할 것)
    """
    )

    llm = ChatOpenAI(model="gpt-4o", temperature=0)
    selection_chain = LLMChain(llm=llm, prompt=selection_prompt)
    selection_result = selection_chain.run(user_input=user_input, case_summaries=case_summaries)

    # 사건 ID 파싱 및 선택
    match = re.search(r"사건 ID[:：]?\s*(차\d{1,2}-\d{1,2})", selection_result)
    selected_id = match.group(1) if match else None
    selected_doc = next((doc for doc in case_docs if doc.metadata.get("id") == selected_id), None)

    # 최종 판단 GPT 프롬프트(선택한 사건 object 내에서 과실비율 판단)
    if selected_doc:
        # 해당 사건 관련 보조 문서들을 함께 전달
        related_docs = [doc for doc in all_docs if selected_id in doc.page_content]
        context_str = "\n\n".join(doc.page_content for doc in related_docs)

        final_prompt = PromptTemplate(
            input_variables=["user_input", "case_data"],
            template="""
    너는 교통사고 과실 판단 전문가야.
    아래 '사고 상황'을 분석하여 핵심 요소를 구조화하고, 반드시 문서 내에서 가장 유사한 사례(case)를 찾아 과실비율을 판단해줘.

    ---

    사고 상황 원문:
    {user_input}

    ➤ 사고 상황 요약 (다음 항목 기준):
    - A차량 신호 및 진행 방식:
    - B차량 신호 및 진행 방식:
    - 충돌 방식 및 위치:
    - 교차로/신호기 유무 등 도로 환경:

    문서:
    {case_data}

    출력 형식 (고정):
    1. 과실비율: A차량 xx% vs B차량 xx%
    2. 판단 근거 요약
    3. 적용 법률:
    - [법률명] 제[조]조 [항]
    4. 참고 판례:
    - [법원명] [사건번호]

    조건:
    - 반드시 문서 내 유사 사례를 기반으로 판단해야 해.
    - 유사 사례와 현재 사고 상황이 정확히 일치하지 않으면, 차이점을 명시하고 과실비율 조정 이유를 설명해.
    - 추측이나 상식은 사용하지 말고, 문서 정보만을 기반으로 판단해.
    """
        )

        final_chain = LLMChain(llm=llm, prompt=final_prompt)
        final_result = final_chain.run(user_input=user_input, case_data=context_str)

        print(f"\n선택된 사건 ID: {selected_id}")
        print("GPT 최종 판단 결과:\n")
        print(final_result)

    else:
        print("\n❌ 사건 ID를 정확히 선택하지 못했습니다.")
        print("GPT 응답:\n", selection_result)

# 함수 정의 (용어/판례 입력시)
def precedent_result(user_input):
    from langchain.prompts import PromptTemplate
    from langchain.chains import RetrievalQA
    from langchain_openai import ChatOpenAI
    from langchain.chains.query_constructor.base import AttributeInfo
    from langchain.retrievers import SelfQueryRetriever

    llm = ChatOpenAI(model="gpt-4o", temperature=0)    

    # 메타데이터 필드 정의 (필수!)
    metadata_field_info = [
        AttributeInfo(
            name="court",
            description="판례의 법원명 (예: 대법원, 서울고등법원 등)",
            type="string"
        ),
        AttributeInfo(
            name="case_id",
            description="사건번호 (예: 92도2077)",
            type="string"
        ),
        AttributeInfo(
            name="term",
            description="용어",
            type="string"
        )
    ]

    # SelfQueryRetriever 생성 (metadata_field_info 필수)
    self_retriever = SelfQueryRetriever.from_llm(
        llm=llm,
        vectorstore=vectorstore,
        document_contents="교통사고 판례 데이터",
        metadata_field_info=metadata_field_info  # ✅ 반드시 필요
    )


    # 프롬프트 구성
    prompt = PromptTemplate(
        input_variables=["question", "context"],
        template="""아래 문서 내용을 바탕으로 사용자가 물어본 용어나 법률 조항, 판례에 대해 정확하고 간결하게 설명해 주세요.
        
        질문: {question}
        
        문서: {context}

        답변 형식:
        - 용어/조항 정의: [정확한 설명]
        - 출처가 명시된 경우: 관련 법률/조문 번호/판례명을 반드시 포함

        답변:
        """
    )

    # QA 체인 구성 및 실행
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=self_retriever,
        chain_type="stuff",
        chain_type_kwargs={"prompt": prompt}
    )

    result = qa_chain.invoke({"query": user_input})
    return f"[판례 설명 결과]\n{result['result']}" 

routing_prompt = PromptTemplate.from_template("""
너는 교통사고 전문 AI 비서야.

다음 사용자 질문을 읽고, 아래 중 어떤 기능을 사용해야 할지 하나만 골라줘:
- 사고상황 과실비율 판단: "function_1"
- 판례에 대한 설명 및용어 설명: "function_2"

반드시 아래 형식으로만 대답해:
[선택된 함수]: function_X
[선택 이유]: (간단한 이유 설명)

질문: {user_input}
""")

# 라우팅 함수
def route_and_respond(user_input: str, all_docs: list) -> str:
    llm = ChatOpenAI(model="gpt-4", temperature=0)
    routing_chain = LLMChain(llm=llm, prompt=routing_prompt)
    result = routing_chain.run(user_input=user_input)

    match = re.search(r"function_\d", result)
    if match:
        chosen = match.group()
        if chosen == "function_1":
            return assess_accident_fault(user_input, all_docs)
        elif chosen == "function_2":
            return precedent_result(user_input)
    else:
        return f"❌ 기능 분류 실패.\nGPT 응답: {result}"
    

# 프로그램 실행
if __name__ == "__main__":
    print("🚗 교통사고 AI 분석기입니다.")
    print("사고 상황이나 알고 싶은 법률/판례 정보를 입력해 주세요.")
    user_input = input("입력 > ").strip()

    if user_input:
        result = route_and_respond(user_input, all_docs)
        print("\n📘 결과 출력:\n")
        print(result)
    else:
        print("❌ 입력이 비어있습니다. 프로그램을 종료합니다.")


🚗 교통사고 AI 분석기입니다.
사고 상황이나 알고 싶은 법률/판례 정보를 입력해 주세요.

📘 결과 출력:

[판례 설명 결과]
- 용어/조항 정의: 동일폭 교차로와 대로/소로 교차로는 도로의 폭에 따라 교차로에서의 과실 여부 및 가해자와 피해자의 구분이 달라질 수 있는 중요한 요소입니다. 대로와 소로의 구분은 도로의 폭을 기준으로 하며, 이는 상대적인 개념입니다. 대로와 소로의 구분은 엄격하게 적용되어야 하며, 진행한 도로를 기준으로 하고, 계측으로 구분하는 것이 아니라 운전자가 일견 분별할 수 있어야 합니다.
- 출처가 명시된 경우: 판례(대법원 97다14187)
